In [1]:
import pandas as pd
import glob
import os
import numpy as np
from time import time
import logging
import gensim
import bz2
import PyPDF2
import requests
import json

/Users/Barry/anaconda/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [15]:
dataset = []

path = '../datas/scrap/person/'
allFiles = glob.glob(os.path.join(path, '*.csv'))

# Load and concactenate all the files into one dataset.
for file_ in allFiles:
    data = pd.read_csv(file_)
    dataset += [data]
datas = pd.concat(dataset)

print(len(data.columns))  
#print('Length of the dataset', len(dataset))

22


In [16]:
datas = datas.dropna(axis = 0, subset = ["PersonIdCode"])

In [13]:
ids = datas.PersonIdCode.unique().astype(int)


In [19]:
url_portrait = "https://www.parlament.ch/sitecollectionimages/profil/portrait-260/"
for id in ids:
    print(id)
    f = open('../datas/pictures/' + str(id) + '.jpg','wb')
    f.write(requests.get(url_portrait + str(id) + '.jpg').content)
    f.close()

2200
2002
2004
2005
2008
2268
2269
2011
2335
2202
2264
2270
2260
2015
2271
2336
2016
2017
2337
2018
2019
2021
2272
2273
2274
2025
2275
2250
2026
2276
2277
2027
2029
2030
2031
2032
2279
2204
2033
2205
2280
2035
2036
2281
2037
2206
2038
2039
2247
2207
2040
2041
2282
2042
2283
2208
2284
2044
2046
2047
2210
2049
2285
2258
2051
2053
2286
2055
2287
2058
2288
2289
2290
2064
2063
2065
2211
2066
2067
2338
2291
2069
2212
2253
2339
2292
2070
2293
2294
2295
2073
2296
2266
2075
2077
2254
2078
2079
2080
2298
2082
2267
2299
2083
2084
2085
2086
2300
2089
2216
2218
2092
2301
2220
2094
2302
2303
2096
2304
2305
2222
2098
2223
2100
2263
2306
2101
2102
2103
2104
2106
2108
2109
2307
2308
2110
2309
2113
2114
2115
2310
2117
2227
2265
2261
2311
2312
2340
2313
2123
2124
2314
2125
2126
2130
2229
2135
2136
2137
2138
2230
2139
2341
2315
2316
2142
2143
2147
2232
2233
2256
2317
2148
2234
2149
2235
2150
2151
2152
2154
2156
2318
2158
2259
2159
2160
2319
2320
2257
2236
2161
2162
2321
2163
2237
2164
2165
2166
2239
2168


In [109]:
datas[datas["FirstName"] == "Peter"]

,Unnamed: 0,DateOfBirth,DateOfDeath,FirstName,GenderAsString,ID,Language,LastName,MaritalStatus,MaritalStatusText,...,NativeLanguage,NumberOfChildren,OfficialName,PersonIdCode,PersonNumber,PlaceOfBirthCanton,PlaceOfBirthCity,Title,TitleText,id
6,6,1942-09-03T00:00:00,NaN,Peter,m,10,FR,Baumberger,2.0,marié(e),...,D,2.0,Baumberger Peter,2269.0,10,Zurich,Winterthur,6.0,Dr. iur.,Peter Baumberger
12,12,1939-06-01T00:00:00,NaN,Peter,m,16,FR,Bircher,2.0,marié(e),...,D,2.0,Bircher Peter,2260.0,16,Argovie,Wölflinswil,NaN,NaN,Peter Bircher
18,18,1933-09-16T00:00:00,NaN,Peter,m,22,FR,Bloetzer,1.0,célibataire,...,D,NaN,Bloetzer,2337.0,22,Valais,Brigue,95.0,dipl. Bauing. ETH,Peter Bloetzer
19,19,1952-03-30T00:00:00,NaN,Peter,m,23,FR,Bodenmann,NaN,NaN,...,NaN,NaN,Bodenmann Peter,2018.0,23,NaN,NaN,NaN,NaN,Peter Bodenmann
105,105,1948-05-14T00:00:00,NaN,Peter,m,111,FR,Hess,2.0,marié(e),...,D,4.0,Hess Peter,2085.0,111,Zoug,Unterägeri,6.0,Dr. iur.,Peter Hess
115,115,1957-09-26T00:00:00,NaN,Peter,m,121,FR,Jenni,NaN,NaN,...,D,NaN,Jenni,2302.0,121,NaN,NaN,NaN,NaN,Peter Jenni
190,190,1940-10-04T00:00:00,NaN,Peter,m,196,FR,Schmid,NaN,NaN,...,NaN,NaN,Schmid Peter,2161.0,196,NaN,NaN,NaN,NaN,Peter Schmid
218,218,1940-04-26T00:00:00,2009-07-04T00:00:00,Peter,m,225,FR,Tschopp,2.0,marié(e),...,F,2.0,Tschopp Peter,2329.0,225,Bâle-Ville,Bâle,91.0,Doctorat,Peter Tschopp
222,222,1946-04-14T00:00:00,NaN,Peter,m,229,FR,Vollmer,2.0,marié(e),...,D,3.0,Vollmer,2251.0,229,Berne,Berne,26.0,Dr. rer. pol.,Peter Vollmer
265,265,1952-06-21T00:00:00,NaN,Peter,m,277,FR,Bieri,2.0,marié(e),...,D,4.0,Bieri,2356.0,277,Zurich,Winterthur,226.0,Dipl. Ing. Agr. ETH / Dr. sc. techn.,Peter Bieri


In [110]:
datas['id'] = datas['FirstName']+' '+datas['LastName']
links = datas[["PersonIdCode"  ,"id"]]

In [111]:
links[links['id']== "Peter Schilliger"]

,PersonIdCode,id
164,3029.0,Peter Schilliger


In [112]:
dictionary = dict(zip(datas["id"].values.tolist(),datas["PersonIdCode"].astype(int).values.tolist()))

In [113]:
dictionary

{'Kurt Bürer': 1005,
 'Paul Wyss': 2191,
 'Serge Beck': 2461,
 'Jean-Claude Rennwald': 2413,
 'Flavio Cotti': 1082,
 'Hubert Bonvin': 2020,
 'Fritz Stalder': 2297,
 'Elisabeth Blunschy-Steiner': 1002,
 'Isabelle Chevalley': 3604,
 'Valentine Friedli': 1022,
 'Maximilian Reimann': 2147,
 'Theo Maissen': 2359,
 'Rudolf jun. Reichling': 2145,
 'Duri Campell': 3066,
 'Christoph Blocher': 2017,
 'Judith Stamm': 2173,
 'Hans Ulrich Baumberger': 1151,
 'Thomas Burgener': 2450,
 'Jean-Nils de Dardel': 2282,
 'Jean Christophe Schwaab': 3020,
 'Jacqueline Badran': 2762,
 'Christine Egerszegi-Obrist': 2380,
 'Werner Hösli': 3047,
 'Pierre Gassmann': 1153,
 'Lisa Mazzone': 3086,
 'Yvette Estermann': 2679,
 'Daniela Schneeberger': 2766,
 'Jean-Pierre Bonny': 2019,
 'Leo Müller': 3012,
 'Werner Carobbio': 2032,
 'Ursula Haller Vannini': 2496,
 'Walter Hess': 2499,
 'Walter Decurtins': 2482,
 'Marco Borradori': 2273,
 'Louis Schelbert': 2654,
 'Martina Munz': 3036,
 'Ernst Leuenberger': 2102,
 'Thoma

In [25]:
links.to_json("linkspersons.json")

In [107]:
with open('linkspersons.json', 'w') as outfile:
    json.dump(dictionary, outfile)